In [ ]:
#Modelling the random forest, gradient boost, MLP Classifier and the LogisticRegression

In [227]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import scale
import matplotlib.pyplot as plt 
from prettytable import PrettyTable
import operator
import xgboost as xgb
from sklearn.metrics import f1_score


In [228]:
my_data = pd.read_csv("C:/Users/Jobin Binoy George/Downloads/Prem_Data/updateddata.csv")

allteams = {}
for club in my_data.groupby('HomeTeam').mean().T.columns:
    allteams[club] = 0
all_fresults = my_data['FTR']
cols_needed = ['HomeGS','AwayGS','HomeGC','AwayGC',
               'HomePoints','AwayPoints','HomeTeamPointsLast5',
               'AwayTeamPointsLast5','PosDif','PointsDif','CLPosDif','FormDif']

inputframe = np.zeros(shape = (len(cols_needed),len(my_data)))
for x in range(len(cols_needed)):
    inputframe[x] = (my_data[cols_needed[x]])
all_finput = pd.DataFrame(data = inputframe.transpose(),columns = cols_needed)
for column in cols_needed:
    all_finput[column] = scale(all_finput[column])

finalinputframe = pd.DataFrame(index = all_finput.index)
for feature, data in all_finput.iteritems():
    if data.dtype == object:
        data = pd.get_dummies(data, prefix = feature)
    
    finalinputframe = finalinputframe.join(data)
all_finput = finalinputframe

In [212]:
xtrain = all_finput.loc[0:5699,:]
ytrain = all_fresults[0:5700]
xtest = all_finput.loc[5700:6080,:]
ytest = all_fresults[5700:6080]



In [213]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

In [184]:
param_grid = [
        {
            'activation' : ['identity', 'logistic', 'tanh', 'relu'],
            'solver' : ['lbfgs', 'sgd', 'adam'],
            'max_iter' : [800],
            'hidden_layer_sizes': [
             (1,),(2,),(3,),(4,),(5,),(6,),(7,),(8,),(9,),(10,),(11,), (12,),(13,),(14,),(15,),(16,),(17,),(18,),(19,),(20,),(21,)
             ]
        }
       ]
clf = GridSearchCV(MLPClassifier(), param_grid, cv=3,
                           scoring='accuracy')
clf.fit(xtrain,ytrain)


print("Best parameters set found on development set:")
print(clf.best_params_)

C:\Users\Jobin Binoy George\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:471: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\Jobin Binoy George\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:471: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\Jobin Binoy George\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:471: ConvergenceWarning: lbfgs failed to conv

KeyboardInterrupt: 

In [217]:
mlp=MLPClassifier(hidden_layer_sizes=(50,20,10),activation='relu',solver='sgd', random_state = 0, max_iter=800)
mlp.fit(x_train,y_train)

Yout=mlp.predict(xtest)
f1_score(ytest,Yout,average='weighted')

0.5640953119076138

In [218]:
for i in range(len(Yout)):
    if Yout[i] == 'H':
        allteams[my_data.loc[5700+i,"HomeTeam"]]+=3
    if Yout[i] == 'A':
        allteams[my_data.loc[5700+i,"AwayTeam"]]+=3
    if Yout[i] == 'D':
        allteams[my_data.loc[5700+i,"AwayTeam"]]+=1
        allteams[my_data.loc[5700+i,"HomeTeam"]]+=1
sorted_teams = sorted( ((value,key) for (key,value) in allteams.items()), reverse = True)
x = PrettyTable()
x.field_names = ["Rank", "Club", "Points"]
for i in range(len(allteams)):
    x.add_row([i+1,sorted_teams[i][1], sorted_teams[i][0]])
    if i == 19:
        break
print(x)
        

+------+----------------+--------+
| Rank |      Club      | Points |
+------+----------------+--------+
|  1   |    Man City    |  103   |
|  2   |   Leicester    |   99   |
|  3   |    Arsenal     |   92   |
|  4   |   Man United   |   82   |
|  5   |    West Ham    |   78   |
|  6   |   Tottenham    |   77   |
|  7   |   Liverpool    |   73   |
|  8   | Crystal Palace |   73   |
|  9   |    Everton     |   61   |
|  10  |    Watford     |   57   |
|  11  |  Southampton   |   54   |
|  12  |   West Brom    |   47   |
|  13  |     Stoke      |   44   |
|  14  |    Chelsea     |   39   |
|  15  |    Swansea     |   38   |
|  16  |  Bournemouth   |   32   |
|  17  |    Norwich     |   27   |
|  18  |   Newcastle    |   22   |
|  19  |   Sunderland   |   11   |
|  20  |  Aston Villa   |   5    |
+------+----------------+--------+


In [221]:
xtrain = all_finput.loc[0:5699,:]
ytrain = all_fresults[0:5700]
xtest = all_finput.loc[5700:6080,:]
ytest = all_fresults[5700:6080]
boost_clf = xgb.XGBClassifier(seed = 82)
boost_clf.fit(xtrain, ytrain)
Yout = boost_clf.predict(xtest)

f1_score(ytest,Yout,average='weighted')

C:\Users\Jobin Binoy George\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[05:16:25] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.5394028034556239

In [222]:
for i in range(len(Yout)):
    if Yout[i] == 'H':
        allteams[my_data.loc[5700+i,"HomeTeam"]]+=3
    if Yout[i] == 'A':
        allteams[my_data.loc[5700+i,"AwayTeam"]]+=3
    if Yout[i] == 'D':
        allteams[my_data.loc[5700+i,"AwayTeam"]]+=1
        allteams[my_data.loc[5700+i,"HomeTeam"]]+=1
sorted_teams = sorted( ((value,key) for (key,value) in allteams.items()), reverse = True)
x = PrettyTable()
x.field_names = ["Rank", "Club", "Points"]
for i in range(len(allteams)):
    x.add_row([i+1,sorted_teams[i][1], sorted_teams[i][0]])
    if i == 19:
        break
print(x)

+------+----------------+--------+
| Rank |      Club      | Points |
+------+----------------+--------+
|  1   |   Leicester    |   90   |
|  2   |    Man City    |   89   |
|  3   |    Arsenal     |   80   |
|  4   |   Man United   |   75   |
|  5   |    West Ham    |   74   |
|  6   |   Tottenham    |   72   |
|  7   |   Liverpool    |   69   |
|  8   |    Everton     |   63   |
|  9   | Crystal Palace |   61   |
|  10  |  Southampton   |   58   |
|  11  |    Chelsea     |   54   |
|  12  |    Watford     |   48   |
|  13  |   West Brom    |   46   |
|  14  |     Stoke      |   43   |
|  15  |    Swansea     |   42   |
|  16  |  Bournemouth   |   30   |
|  17  |    Norwich     |   26   |
|  18  |   Sunderland   |   24   |
|  19  |   Newcastle    |   24   |
|  20  |  Aston Villa   |   7    |
+------+----------------+--------+


In [225]:
xtrain = all_finput.loc[0:5699,:]
ytrain = all_fresults[0:5700]
xtest = all_finput.loc[5700:6080,:]
ytest = all_fresults[5700:6080]
forest_clf = RandomForestClassifier(n_estimators = 100, max_depth = 20, random_state = 0)
forest_clf.fit(xtrain, ytrain)
Yout = forest_clf.predict(xtest)
f1_score(ytest,Yout,average='weighted')

0.573435547472856

In [226]:
for i in range(len(Yout)):
    if Yout[i] == 'H':
        allteams[my_data.loc[5700+i,"HomeTeam"]]+=3
    if Yout[i] == 'A':
        allteams[my_data.loc[5700+i,"AwayTeam"]]+=3
    if Yout[i] == 'D':
        allteams[my_data.loc[5700+i,"AwayTeam"]]+=1
        allteams[my_data.loc[5700+i,"HomeTeam"]]+=1
sorted_teams = sorted( ((value,key) for (key,value) in allteams.items()), reverse = True)
x = PrettyTable()
x.field_names = ["Rank", "Club", "Points"]
for i in range(len(allteams)):
    x.add_row([i+1,sorted_teams[i][1], sorted_teams[i][0]])
    if i == 19:
        break
print(x)

+------+----------------+--------+
| Rank |      Club      | Points |
+------+----------------+--------+
|  1   |    Man City    |   96   |
|  2   |   Leicester    |   91   |
|  3   |    Arsenal     |   88   |
|  4   |   Man United   |   81   |
|  5   |   Liverpool    |   80   |
|  6   |   Tottenham    |   79   |
|  7   |    West Ham    |   70   |
|  8   |    Everton     |   66   |
|  9   | Crystal Palace |   63   |
|  10  |  Southampton   |   59   |
|  11  |    Watford     |   51   |
|  12  |    Chelsea     |   51   |
|  13  |   West Brom    |   46   |
|  14  |    Swansea     |   35   |
|  15  |     Stoke      |   35   |
|  16  |  Bournemouth   |   27   |
|  17  |   Newcastle    |   24   |
|  18  |    Norwich     |   23   |
|  19  |   Sunderland   |   18   |
|  20  |  Aston Villa   |   6    |
+------+----------------+--------+


In [229]:
xtrain = all_finput.loc[0:5699,:]
ytrain = all_fresults[0:5700]
xtest = all_finput.loc[5700:6080,:]
ytest = all_fresults[5700:6080]
Logistic_clf = LogisticRegression(random_state = 42)
Logistic_clf.fit(xtrain, ytrain)
Yout = Logistic_clf.predict(xtest)
f1_score(ytest,Yout,average='weighted')

0.5693551470255247

In [230]:
for i in range(len(Yout)):
    if Yout[i] == 'H':
        allteams[my_data.loc[5700+i,"HomeTeam"]]+=3
    if Yout[i] == 'A':
        allteams[my_data.loc[5700+i,"AwayTeam"]]+=3
    if Yout[i] == 'D':
        allteams[my_data.loc[5700+i,"AwayTeam"]]+=1
        allteams[my_data.loc[5700+i,"HomeTeam"]]+=1
sorted_teams = sorted( ((value,key) for (key,value) in allteams.items()), reverse = True)
x = PrettyTable()
x.field_names = ["Rank", "Club", "Points"]
for i in range(len(allteams)):
    x.add_row([i+1,sorted_teams[i][1], sorted_teams[i][0]])
    if i == 19:
        break
print(x)

+------+----------------+--------+
| Rank |      Club      | Points |
+------+----------------+--------+
|  1   |   Leicester    |  101   |
|  2   |    Man City    |  100   |
|  3   |    Arsenal     |   91   |
|  4   |   Man United   |   86   |
|  5   |   Tottenham    |   82   |
|  6   |    West Ham    |   79   |
|  7   | Crystal Palace |   69   |
|  8   |   Liverpool    |   68   |
|  9   |    Everton     |   65   |
|  10  |  Southampton   |   61   |
|  11  |    Watford     |   52   |
|  12  |   West Brom    |   44   |
|  13  |     Stoke      |   41   |
|  14  |    Chelsea     |   38   |
|  15  |    Swansea     |   36   |
|  16  |  Bournemouth   |   31   |
|  17  |    Norwich     |   27   |
|  18  |   Newcastle    |   22   |
|  19  |   Sunderland   |   14   |
|  20  |  Aston Villa   |   5    |
+------+----------------+--------+
